In [ ]:
import ftplib
import os
import csv

# FTP server details
ftp_server = 'ftp.ebi.ac.uk'
ftp_directory = '/pub/databases/mett/annotations/v1_2024-04-15/'

# Connect to the FTP server
ftp = ftplib.FTP(ftp_server)
ftp.login()

# Change to the target directory
ftp.cwd(ftp_directory)

# List directories (isolates)
isolates = ftp.nlst()

# Prepare the CSV data
# csv_data = [["Isolate Name", "Assembly Name", "FTP Path"]]
csv_data = [["Isolate Name", "FTP File"]]

# Function to list files in the directory and subdirectories
def list_files(ftp, path):
    file_list = []
    try:
        file_list = ftp.nlst(path)
    except ftplib.error_perm as resp:
        if str(resp) == "550 No files found":
            pass
        else:
            raise
    return file_list

# Iterate through each isolate directory
for isolate in isolates:
    isolate_path = os.path.join(ftp_directory, isolate, 'functional_annotation/merged_gff/')
    gff_files = list_files(ftp, isolate_path)
    
    for gff_file in gff_files:
        if gff_file.endswith('_annotations.gff'):
            # print(f'check --- {os.path.basename(gff_file)}')
            ftp_file_name = os.path.basename(gff_file)
            # assembly_name = os.path.splitext(os.path.basename(gff_file))[0]
            # isolate_name = isolate
            # ftp_path = f'http://{ftp_server}{gff_file}'
            # ftp_path = gff_file
            
            csv_data.append([isolate, ftp_file_name])

# Save to CSV file
csv_filename = 'gff_files.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(csv_data)

print(f"CSV file '{csv_filename}' has been created.")

# Close the FTP connection
ftp.quit()
